# Resume Extraction

## 1. Environment Setup

All these commands run in terminal (Ubuntu 20.04)

Upgrading pip 

### 1.1 PaddleOCR

* **<font color=red>Installing PaddlePaddle</font>**

Before executing next cell please check this link

https://www.paddlepaddle.org.cn/en/install/quick?docurl=/documentation/docs/en/install/pip/linux-pip_en.html

Installing PaddlePaddle

* **<font color=red>Installing PaddleOCR</font>**

Installing Git

Clone PaddleOCR repo

Get into PaddleOCR folder

Installing PaddleOCR's requirements

In case you face any problem in protobuf version

Installing PaddleOCR

### 1.2 LayoutParser

* **<font color=red>Installing the LayoutParser main library</font>**

Before executing next cell please check this link

https://layout-parser.github.io/tutorials/installation

* **<font color=red>Installing Detectron2 for Using Layout Models</font>**

Before executing next cell please check this link

https://github.com/facebookresearch/detectron2/blob/main/INSTALL.md

Installing Detectron2's requirements

Build Detectron2 from Source

* **<font color=red>Installing Tesseract</font>**

Before executing next cell please check this link

https://tesseract-ocr.github.io/tessdoc/Installation.html

Installing OCR utils

Installing Tesseract

### If you get this error : No module named 'tools.infer' 

check this for debugging : https://github.com/PaddlePaddle/PaddleOCR/issues/1024#issuecomment-1105946934

You will find that you need to open this 2 files and change "tools" by "paddleocr.tools" :

/usr/local/lib/python3.8/dist-packages/paddleocr/paddleocr.py

/usr/local/lib/python3.8/dist-packages/paddleocr/tools/infer/predict_system.py

### 1.3 Googletrans

### 1.4 SpaCy

### 1.5 SkillNer

### 1.6 Stanza

## 2. Resume Extraction

### 2.1 Resume Preparation

Converting the resume from pdf to image file

In [ ]:
pdf_path = './cv7.pdf'
img_name = pdf_path[2:-4] + '.jpg'

from pdf2image import convert_from_path
 
images = convert_from_path(pdf_path)

for i in range(len(images)) : images[i].save(img_name, 'JPEG')

Load the resume image

In [ ]:
from PIL import Image
image = Image.open(img_name)

Dispaly the resume image

In [ ]:
image.show()

### 2.2 Resume Text Extraction

#### 2.2.1 Layout Analysis

* **<font color=red>PaddleOCR</font>**

In this section we will use PaddleOCR as a text detector due to its high detection accuracy compared to LayoutParser

Detecting boxes texts by position from resume using PaddleOCR

In [ ]:
from paddleocr import PaddleOCR,draw_ocr

ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False)

paddleOCR_result = ocr.ocr(img_name, cls=True)

In [ ]:
for x in paddleOCR_result : print(x, end="\n\n")

Displaying paddleOCR's result in image

In [ ]:
from PIL import Image

image = Image.open(img_path).convert('RGB')

boxes  = [line[0] for line in paddleOCR_result]
txts   = [line[1][0] for line in paddleOCR_result]
scores = [line[1][1] for line in paddleOCR_result]

im_show = draw_ocr(image, boxes, txts, scores, font_path='/home/yns/PaddleOCR/doc/fonts/simfang.ttf')
im_show = Image.fromarray(im_show)
#im_show.save('paddleOCR_result.jpg')
im_show.show()

* **<font color=red>LayoutParser</font>**

Next, we will use LayoutParser for resume segmentation to find the most probable layout boxes that divide resume meaningfully using two pre-trained model

In [ ]:
import layoutparser as lp
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Converting image to array list

In [ ]:
img = np.asarray(image)

Detecting layouts by position from resume using First model (Mask R-CNN pre-trained on PubLayNet)

In [ ]:
model_1 = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
layout_result_1 = model_1.detect(img)

Displaying layout_result_1

In [ ]:
layout_result_1

Filtring and displaying layout_result_1 in image

In [ ]:
layout_result_1 = lp.Layout([b for b in layout_result_1 if b.type in ('Text','Title')])
lp.draw_box(img, layout_result_1,  box_width=2, box_alpha=0.2, show_element_type=False)

Detecting layouts by position from resume using First model (Mask R-CNN pre-trained on PrimaLayout)

In [ ]:
model_2 = lp.Detectron2LayoutModel('lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
                                 label_map={1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"})
layout_result_2 = model_2.detect(img)

Displaying layout_result_2

In [ ]:
layout_result_2

Filtring and displaying layout_result_2 in image

In [ ]:
layout_result_2 = lp.Layout([b for b in layout_result_2 if b.type == 'TextRegion'])
lp.draw_box(img, layout_result_2,  box_width=2, box_alpha=0.2, show_element_type=False)

Converting paddleOCR_result to LayoutParser Class

In [ ]:
paddle_layout = lp.elements.layout.Layout()

for res in paddleOCR_result:
    
    box  = res[0]
    text = res[1][0]
    
    x_1 = max(box[0][0], box[3][0])

    y_1 = max(box[0][1], box[1][1])

    x_2 = max(box[1][0], box[2][0])

    y_2 = max(box[2][1], box[3][1])

    Rectangle = lp.elements.layout_elements.Rectangle(x_1, y_1, x_2, y_2)

    TextBlock = lp.elements.layout_elements.TextBlock(Rectangle)
    
    TextBlock.text = text

    paddle_layout._blocks.append(TextBlock)

Displaying paddleOCR_result in image

In [ ]:
lp.draw_box(img, paddle_layout,  box_width=2, box_alpha=0.2, show_element_type=False)

Integrating all results (1st model + 2nd model + paddleOCR) in one layout_result

In [ ]:
layout_result = layout_result_1 + layout_result_2 + paddle_layout

Creating a function for consolidating and removing redundant boxes

In [ ]:
import torch
import torchvision.ops.boxes as bops

def set_coordinate(data):

  x1 = data.block.x_1
  y1 = data.block.y_1
  x2 = data.block.x_2
  y2 = data.block.y_2

  return torch.tensor([[x1, y1, x2, y2]], dtype=torch.float)


def check_intersection(block_1, block_2):
    
    bb1 = set_coordinate(block_1)
    bb2 = set_coordinate(block_2)

    iou = bops.box_iou(bb1, bb2)

    if iou.tolist()[0][0] != 0:
        return True
    
    else :
        return False
    
    
def union(layout_result):
    
    for layout_i in layout_result:
        for layout_j in layout_result:
            if layout_i == layout_j == layout_result[-1]:
                return layout_result
            if layout_i != layout_j:
                if check_intersection(layout_i, layout_j) == True:
                    layout_result.insert(layout_result.index(layout_i),layout_i.union(layout_j,strict=False))
                    layout_result.remove(layout_i)
                    layout_result.remove(layout_j)
                    return union(layout_result)    

In [ ]:
layout_result = union(layout_result)

Displaying the final in image

In [ ]:
lp.draw_box(img, layout_result,  box_width=2, box_alpha=0.2, show_element_type=False)

#### 2.2.2 Text Extraction

After detecting resume's layouts, we need to put them in order before extracting their texts

In [ ]:
text_blocks = lp.Layout([b for b in layout_result])

image_width = len(img[0])

# Split the resume into 2 column left and right based on x1 coordinate
left_blocks  = [b for b in text_blocks if b.block.x_1 <= image_width/4]
# Sort element ID of the left column based on y1 coordinate
left_blocks.sort(key = lambda b:b.block.y_1)


right_blocks = [b for b in text_blocks if b not in left_blocks]
# Sort element ID of the right column based on y1 coordinate
right_blocks.sort(key = lambda b:b.block.y_1)

# Sort the overall element ID starts from left column
text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])


Calling OCR agent (Tesseract) to extract texts from resume's layouts

In [ ]:
ocr_agent = lp.TesseractAgent("eng")

Exclude paddleOCR boxes which not affected by layoutParser union due to high quality of the extracted text by PaddleOCR. As a result we keep the original text of PaddleOCR_result

In [ ]:
for block in text_blocks:
    
    # Exclude PaddleOCR_result boxes which their texts already extracted
    if block.text == None :

        # Crop image around the detected layout
        segment_image = (block
                           .pad(left=15, right=15, top=5, bottom=5)
                           .crop_image(img))

        # Perform OCR
        text = ocr_agent.detect(segment_image)

        # Save OCR result
        block.set(text=text, inplace=True)

Saving Layouts's texts in one list

In [ ]:
cv = [b.text for b in text_blocks]

Cleaning extracted texts

In [ ]:
import re
for i in range(len(cv)) :
    cv[i] = re.sub('\n', ' ', cv[i])
    cv[i] = re.sub('\x0c', '', cv[i])
    cv[i] = re.sub('  ', ' ', cv[i])
    cv[i] = re.sub(' \.', '.', cv[i])
    cv[i] = re.sub('[%s]' % re.escape("""!"#$%&'()*+:;\<=>?[]^`{|}~"""), ' ', cv[i])
    cv[i] = cv[i].strip()

Merging all texts in one text

In [ ]:
resume = ""
for x in cv :
    resume = resume + x + " "
resume=resume[:-1]
resume

Translating resume from french to english

In [ ]:
from googletrans import Translator

translator = Translator()
for i in range(len(resume)):
    try :
        f = translator.detect(resume[i])
        if f.lang == 'fr':
            t = translator.translate(resume[i], src='fr', dest='en')
            resume[i] = t.text
            print(resume[i],end='\n\n')
        elif f.lang == 'en':
            t = translator.translate(resume[i], src='fr', dest='en')
            resume[i] = t.text
            print(resume[i],end='\n\n')
        else:
            print(resume[i])
    except : pass
    

#### 2.2.3 Resume Analysis

After extracting the text from the resume and cleaning it up, we will analyze it to find useful informations like skills, personel informations(title, name, location...)

* **<font color=red>Skills Extraction with SkillNer</font>**

SkillNer is an NLP module to automatically Extract skills and certifications from unstructured job postings, texts, and applicant's resumes.

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_trf")

# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# extract skills from resume
annotations = skill_extractor.annotate(resume)

# inspect results by rendering the text with the annotated skills.
skill_extractor.describe(annotations)

Displaying extracted skills with their scores

In [ ]:
skilldic = {}

for skill in annotations['results']['full_matches']:
    skilldic[SKILL_DB[skill['skill_id']]['skill_name']] = (SKILL_DB[skill['skill_id']]['skill_type'],skill["score"])
    
for skill in annotations['results']['ngram_scored']:
    skilldic[SKILL_DB[skill['skill_id']]['skill_name']] = (SKILL_DB[skill['skill_id']]['skill_type'],skill["score"])
    
print('Skill',46*" " ,"Type",10*" ","Score","\n")

for skill in skilldic :
    print(skill, (50-len(skill))*"-", ">" ,skilldic[skill][0], 5*"-",skilldic[skill][1])

* **<font color=red>Named-entity recognition (NER)</font>**

Named-entity recognition (NER) is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

* **<font color=green>NER with SpaCy </font>**

In [ ]:
doc = nlp(resume)

for ent in doc.ents :
    print(ent.text,(50-len(ent.text)) * '-',  ">"  ,ent.label_,"\n")

* **<font color=green>NER with Stanza </font>**

In [ ]:
import stanza
from stanza.server import CoreNLPClient
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

with CoreNLPClient(
        
        annotators = ['tokenize','ssplit','pos','lemma','ner'],
        timeout    = 30000,
        memory     = '10G',
        threads    = '4',
        be_quiet   = True

                            ) as client :
    
    ann = client.annotate(resume,output_format='json')

for sen in ann['sentences']:
    for ent in sen['entitymentions']:
        print(ent['text'],(50-len(ent['text']))*"-", ">" ,ent['ner'],"\n")